In [1]:
import tools.sleeper_read as sr
from tools.sql_cnx import sql_cnx
import pandas as pd
import numpy as np

import importlib

### Build overall player table

Sleeper has player data that updates less often and is not unique to leagues

Pushes data to a data warehouse, in this case Planetscale using sql_cnx, which is a custom SQLAlchemy wrapper made to interface with Planetscale

In [3]:
#build player table and upload it to planetscale

#uncomment below to build:

# import requests

# players = sr.PlayerData()

# instance = sql_cnx(env_path='init/.env')

# instance = sql_cnx(env_path='init/.env')
# conn = instance.cnx
# players.to_sql(name='sleeper_players',con=conn,if_exists='replace')
# instance.cleanup()

# instance.cleanup()

instance = sql_cnx(env_path='init/.env')
conn = instance.cnx
x = pd.read_sql('select * from sleeper_players limit 100',con = conn)
instance.cleanup()

x.head()

,index,fantasy_data_id,status,injury_start_date,years_exp,yahoo_id,practice_description,position,hashtag,gsis_id,...,depth_chart_position,injury_status,search_first_name,full_name,birth_date,birth_country,pandascore_id,active,birth_city,fantasy_positions
0,0,18829.0,Inactive,None,0.0,NaN,None,None,#davecaldwell-NFL-FA-0,None,...,None,None,dave,Dave Caldwell,None,None,None,0,None,None
1,1,19705.0,Active,None,6.0,30715.0,None,WR,#gregward-NFL-PHI-84,00-0033733,...,SWR,None,greg,Greg Ward,1995-07-12,None,None,1,None,['WR']
2,2,15244.0,Inactive,None,7.0,26894.0,None,G,#alvinbailey-NFL-FA-78,None,...,None,None,alvin,Alvin Bailey,1991-08-26,None,None,0,None,['OL']
3,3,13170.0,Injured Reserve,None,11.0,24942.0,None,TE,#nilespaul-NFL-FA-0,00-0028093,...,None,None,niles,Niles Paul,1989-08-09,None,None,0,None,['TE']
4,4,19247.0,Active,None,5.0,30668.0,None,OT,#kwaydemiller-NFL-FA-63,None,...,None,None,kwayde,Kwayde Miller,1993-12-11,None,None,1,None,['OL']


### Test Out sleeper_read.py Package

In [18]:
# initialize league read:
# importlib temporary while working on library

importlib.reload(sr)


<module 'tools.sleeper_read' from 'c:\\Users\\samml\\Documents\\PyProj\\Planetscale\\tools\\sleeper_read.py'>

In [19]:
id = 832718546487357440
data = sr.ffData(id)

print(f'''BUILT TABLES: {[attr for attr in dir(data) if attr.endswith('table')]}
FETCH FUNCS: {[attr for attr in dir(data) if attr.startswith(('get'))]} 
RAW FROM FETCH: {[attr for attr in dir(data) if (not attr.startswith(('__','get','_')) and not attr.endswith('table'))]}
CONST: {[attr for attr in dir(data) if (attr.startswith(('_')) and not attr.startswith(('__')))]}''')


BUILT TABLES: ['free_agent_table', 'matchup_table', 'results_table', 'trade_table', 'transaction_table', 'users_table']
FETCH FUNCS: ['get_league', 'get_matchups', 'get_nfl_state', 'get_rosters', 'get_transactions', 'get_users'] 
RAW FROM FETCH: ['league', 'matchups', 'nfl', 'rosters', 'transactions', 'users']
CONST: ['_current_week', '_playoff_week', '_season']


In [7]:
data.users_table

,league_id,season,user_id,username,roster_id
0,832718546487357440,2022,465633370747957248,LEDRs,1
1,832718546487357440,2022,465636493809938432,smlederer,2
2,832718546487357440,2022,466716381887328256,bbrbr,3
3,832718546487357440,2022,469271575548391424,zgreen1017,4
4,832718546487357440,2022,469319841732358144,aleith,5
5,832718546487357440,2022,469921663765442560,mattdemichiel,6
6,832718546487357440,2022,471147635869216768,Subject2Change,7
7,832718546487357440,2022,472159133173280768,nlfish28,9
8,832718546487357440,2022,472234548525002752,twtytwty,10
9,832718546487357440,2022,728408081750405120,HeMarv,12


In [8]:
data.matchup_table

,league_id,season,week,matchup_id,roster_id_root,user_id_root,username_root,points_root,roster_id_challenger,user_id_challenger,username_challenger,points_challenger
1,832718546487357440,2022,1,2,1,465633370747957248,LEDRs,111.38,10,472234548525002752,twtytwty,143.12
2,832718546487357440,2022,1,2,10,472234548525002752,twtytwty,143.12,1,465633370747957248,LEDRs,111.38
5,832718546487357440,2022,1,6,2,465636493809938432,smlederer,112.50,11,731254033469263872,H1nduHammer,82.60
6,832718546487357440,2022,1,6,11,731254033469263872,H1nduHammer,82.60,2,465636493809938432,smlederer,112.50
9,832718546487357440,2022,1,3,3,466716381887328256,bbrbr,127.86,7,471147635869216768,Subject2Change,91.78
...,...,...,...,...,...,...,...,...,...,...,...,...
326,832718546487357440,2022,14,2,10,472234548525002752,twtytwty,115.96,4,469271575548391424,zgreen1017,96.96
329,832718546487357440,2022,14,3,5,469319841732358144,aleith,102.92,7,471147635869216768,Subject2Change,141.26
330,832718546487357440,2022,14,3,7,471147635869216768,Subject2Change,141.26,5,469319841732358144,aleith,102.92
333,832718546487357440,2022,14,5,9,472159133173280768,nlfish28,93.78,11,731254033469263872,H1nduHammer,78.56


In [9]:
data.results_table

,league_id,week,user_id,username,roster_id,matchup_id,player_id,points,is_starter,season
0,832718546487357440,1,465633370747957248,LEDRs,1,2,SF,7.00,True,2022
1,832718546487357440,1,465633370747957248,LEDRs,1,2,8129,4.40,True,2022
2,832718546487357440,1,465633370747957248,LEDRs,1,2,7525,0.00,True,2022
3,832718546487357440,1,465633370747957248,LEDRs,1,2,6820,20.90,True,2022
4,832718546487357440,1,465633370747957248,LEDRs,1,2,5927,12.80,True,2022
...,...,...,...,...,...,...,...,...,...,...
10,832718546487357440,14,728408081750405120,HeMarv,12,4,4018,11.60,True,2022
11,832718546487357440,14,728408081750405120,HeMarv,12,4,3294,13.66,False,2022
12,832718546487357440,14,728408081750405120,HeMarv,12,4,2749,4.90,True,2022
13,832718546487357440,14,728408081750405120,HeMarv,12,4,1479,15.20,True,2022


In [20]:
data.free_agent_table

,league_id,season,type,week,transaction_id,status,bid,roster_id,player_add,player_drop,notes
0,832718546487357440,2022,waiver,1,875603070862704640,failed,2.0,8,4082,7610,This player was claimed by another owner.
1,832718546487357440,2022,waiver,1,875601951117717504,complete,1.0,2,3423,2431,Your waiver claim was processed successfully!
2,832718546487357440,2022,waiver,1,875598648455643136,complete,5.0,9,2306,5347,Your waiver claim was processed successfully!
3,832718546487357440,2022,waiver,1,875478147561369600,failed,1.0,2,4082,2431,This player was claimed by another owner.
4,832718546487357440,2022,waiver,1,875476326268788736,failed,5.0,2,5284,2431,This player was claimed by another owner.
...,...,...,...,...,...,...,...,...,...,...,...
20,832718546487357440,2022,free_agent,14,906309822289424384,complete,NaN,6,6920,6850,None
21,832718546487357440,2022,free_agent,14,906279744201441280,complete,NaN,2,7594,NE,None
22,832718546487357440,2022,free_agent,14,906228771122720768,complete,NaN,4,PIT,ATL,None
23,832718546487357440,2022,free_agent,14,906203546595241984,complete,NaN,1,5857,5347,None


In [21]:
data.trade_table

,league_id,season,transaction_id,week,player_id,roster_id_to,waiver_amount,roster_id_from
0,832718546487357440,2022,880171262859829248,3,7090,2,None,9
1,832718546487357440,2022,880171262859829248,3,5970,9,None,2
2,832718546487357440,2022,880171262859829248,3,5846,2,None,9
3,832718546487357440,2022,880171262859829248,3,2028,9,None,2
0,832718546487357440,2022,883212353129545728,4,947,3,None,9
1,832718546487357440,2022,883212353129545728,4,7601,9,None,3
2,832718546487357440,2022,883212353129545728,4,7600,3,None,9
3,832718546487357440,2022,883212353129545728,4,4985,9,None,3
4,832718546487357440,2022,883212353129545728,4,4602,9,None,3
5,832718546487357440,2022,883212353129545728,4,2320,3,None,9


In [21]:
#upload to planetscale

instance = sql_cnx('init/.env')
conn = instance.cnx

# upload users
#data.users_table.to_sql(name='users',con=conn, if_exists='replace')

# upload matchups
#data.matchup_table.to_sql(name='matchup',con=conn, if_exists='replace')

# upload results
#data.results_table.to_sql(name='results',con=conn, if_exists='replace')
    
instance.cleanup()

### Example of an end point user using SQL to query from the tables:

In [22]:
#get history of wins from all managers in the league history

instance = sql_cnx('init/.env')
conn = instance.cnx

res = pd.read_sql(sql = '''

select username_root, username_challenger, week, season, matchup_id, points_root, points_challenger,
case when points_root>points_challenger then 'win' else 'loss' end as result
from matchup
''',con = conn)

instance.cleanup()

res.head()

,username_root,username_challenger,week,season,matchup_id,points_root,points_challenger,result
0,LEDRs,twtytwty,1,2022,2,111.38,143.12,loss
1,twtytwty,LEDRs,1,2022,2,143.12,111.38,win
2,smlederer,H1nduHammer,1,2022,6,112.50,82.60,win
3,H1nduHammer,smlederer,1,2022,6,82.60,112.50,loss
4,bbrbr,Subject2Change,1,2022,3,127.86,91.78,win


In [23]:
def wins(x):
    y=np.count_nonzero(x == 'win')
    return y

def losses(x):
    y=np.count_nonzero(x == 'loss')
    return y

res[['username_root','result','points_root']].groupby('username_root').aggregate({'result':[wins,losses],'points_root':['sum','mean']}).sort_values(('result','wins'), ascending=False)

result        points_root            
                 wins losses         sum        mean
username_root                                       
LEDRs              10      4     1735.20  123.942857
smlederer          10      4     1759.94  125.710000
twtytwty           10      4     1674.96  119.640000
Subject2Change      9      5     1633.30  116.664286
mattdemichiel       8      6     1652.24  118.017143
HeMarv              7      7     1530.00  109.285714
aleith              7      7     1516.88  108.348571
H1nduHammer         6      8     1436.10  102.578571
Mnorts              6      8     1646.18  117.584286
nlfish28            5      9     1449.08  103.505714
bbrbr               3     11     1372.98   98.070000
zgreen1017          3     11     1458.08  104.148571

In [24]:
# get the results of a trade since the trade happened in a season, only consider starts:

trade_id = 752195936595365888

instance = sql_cnx('init/.env')
conn = instance.cnx

df = pd.read_sql(f'''select distinct u.username,coalesce(case when sp.full_name = 'nan' then Null else sp.full_name end,tt.player_id) player_name,tt.player_id, r.week, r.points
from test.trades_transactions as tt
join results r on r.player_id = tt.player_id and r.season = tt.season and r.week >= tt.week
join test.sleeper_players as sp on sp.player_id = tt.player_id
join users as u on u.roster_id = tt.roster_id_to
where transaction_id = {trade_id}
and r.is_starter = True''', con = conn)

instance.cleanup()

df.sort_values('week')

,username,player_name,player_id,week,points
0,HeMarv,Leonard Fournette,3969,5,19.00
9,LEDRs,Jalen Hurts,6904,5,22.92
1,HeMarv,Leonard Fournette,3969,6,27.70
10,LEDRs,Jalen Hurts,6904,6,25.00
2,HeMarv,Leonard Fournette,3969,7,16.00
11,LEDRs,Jalen Hurts,6904,7,23.54
3,HeMarv,Leonard Fournette,3969,8,5.80
12,LEDRs,Jalen Hurts,6904,8,11.22
13,LEDRs,Jalen Hurts,6904,9,16.68
4,HeMarv,Leonard Fournette,3969,10,13.20
